# Kwirk 

![](https://upload.wikimedia.org/wikipedia/en/8/80/Kwirk_Cover.png)

# Organización del código en clases

**Este es el primer notebook de para primera práctica obligatoria**.


**Habrá un segundo notebook en el que se consigue que juegue de manera autónoma utilizando A\* **

----------------------------------------


En esta primera parte de la práctica se va a estructurar el código de las funciones básicas (del notebook anterior) en clases para:
1. Poder interactuar con las clases del profesor que controlan el flujo del juego.
2. Dejar el código listo para su adaptación al modo automático usando A\*.

------------------
# Autores:
- **Rodrigo Díaz García**
- **Miguel Barriuso García**



** Para este notebook el profesor proporciona un conjunto de clases que el alumno no tiene que modificar, pero puede hacerlo si lo desea **.

El profesor proporciona:
1. Este notebook con las explicaciones y el *main*.
2. La clase *ui* que implementa los controles (botones, desplegables, representación visual del juego).
3. La clase *Mediator* que pone en contacto los componentes gráficos entre si y utiliza los métodos de la clase *State*, *Level*, *Model* y *Loader*.
4. *State*: Funciones para la construcción y consulta de estados (jugadores, cajas ...). Módulo Kwirk (Kwirk.py)
5. *Level*: Funciones para la construcción y consulta de niveles (tablero, destinos). Módulo Kwirk.


## Descripción de las clases

La representación del estado y de los niveles visto en el notebook de funciones básicas se ha organizado en clases.

La clase State:
- tiene un constructor que recibe los jugadores, las cajas, el agua y el turno del jugador (True mueve el primero, False mueve el segundo). 
- tiene un método **get_players** que devuelve los jugadores.
- tiene un método **get_boxes** que devuelve las cajas.
- tiene un método **get_water** que devuelve las coordenadas con agua.
- tiene un método **get_turn** que devuelve a que jugador le toca mover.

- tiene un método **\_\_str\_\_** que permite obtener la representación en modo texto del objeto y un método **\_\_repr\_\_** equivalente, para usarlo dentro de colecciones.

- Tiene también un método que permite determinar si dos estados son iguales y también otro para calcular su _hash_.

La clase Level:
- tiene un constructor que recibe el tablero y los destinos.
- tiene un método **get_board** que devuelve el tablero. Las paredes y huecos.
- tiene un método **get_destination** que devuelve el destino al que se deben dirigir los jugadores.
- tiene un método **\_\_str\_\_** que permite obtener la representación en modo texto del objeto y un método **\_\_repr\_\_** equivalente.

Loader, UI, Mediator:

In [ ]:
# ejemplo de como se carga un nivel
import os
from LoaderKwirk import Loader
    
l = Loader()
l.get_all_levels()
# os.sep es el separador de ficheros ("/" en linux "\" en windows)
level_txt = open("."+os.sep+"levels"+os.sep+"level_medio.txt",'r').read()

level, state = l.load_level(level_txt)

In [ ]:
level.get_board()

In [ ]:
# ejemplo de como se visualiza un nivel
from UI import gui
from ipywidgets import HTML
ui = gui()
htmlStr = ui.get_html(level, state)
HTML(value = htmlStr)

El alumno debe completar la implementación de los métodos de este notebook:


## Funciones a implementar por el alumno

Nota: En el notebook anterior se usaban variables globales. En este notebook los valores están encapsulados en objetos. Pero la funcionalidad es la misma.

- `move(level,state,mov)`: Recibe un nivel, un estado y un movimiento. **Devuelve un nuevo estado** resultante de aplicar el movimiento, esta función puede reutilizar las funciones del notebook anterior.

- `change_turn(state)`: Recibe un estado. **Devuelve un nuevo estado** con todo igual pero el turno cambiado.

- `is_goal(state)`: Recibe un estado. Devuelve True si los dos jugadores son None (los dos jugadores han llegado a la meta y es un estado final).

No debes borrar `# Cargar Celda`

In [2]:
# Cargar Celda
# NO tocar esta primera linea

# Completa las funciones de esta celda y pruebalas en celdas posteriores
# Reutiliza el código de las funciones básicas que te sirva
from Kwirk import Level, State
import pandas as pd
import numpy as np
import copy


######## main functions
def move(level,state,mov):
    """ Move current player [1,0] up, [-1,0] down, [0,1] rigth, [0,-1] left
    
    Parameters
    ----------
    level -- a level
    state -- a state
    mov -- direction of the movement
    
    Returns
    --------
    Returns a new state or the same if the movment is not possible
    
    """
    player = state.get_players()[not state.get_turn()]
    adversary = state.get_players()[state.get_turn()]
    
    if (player is None):
        return state
    
    result = State(copy.deepcopy(state.get_players()), copy.deepcopy(state.get_boxes()), copy.deepcopy(state.get_water()), copy.copy(state.get_turn()))
    position = [player[dim] + mov[dim] for dim in range(min(len(mov), len(player)))]
    if level.board[position[0]][position[1]] == 1:
        return state
    if adversary is not None and adversary[0] == position[0] and adversary[1] == position[1]:
        return state
    for water in result.get_water():
        if water[0] == position[0] and water[1] == position[1]:
            return state
    for box in result.get_boxes():
        if box[0] == position[0] and box[1] == position[1]:
            box_position = [position[dim] + mov[dim] for dim in range(min(len(mov), len(position)))]
            # Wall
            if level.board[box_position[0]][box_position[1]] == 1:
                return state
            
            # Another player
            if adversary is not None and adversary[0] == box_position[0] and adversary[1] == box_position[1]:
                return state
        
            # Another box
            result.boxes.remove(box)
            if any(box2[0] == box_position[0] and box2[1] == box_position[1] for box2 in result.get_boxes()):
                return state
            
            # Water
            bridge = False
            for water in result.get_water():
                if water[0] == box_position[0] and water[1] == box_position[1]:
                    result.water.remove(water)
                    bridge = True
                    break
            # Grass
            if not bridge:
                result.boxes.add((box_position[0], box_position[1]))
            break
    
    if level.get_destination()[0] == position[0] and level.get_destination()[1] == position[1]:
        result.players[not state.get_turn()] = None
        return change_turn(result)
    
    result.players[not state.get_turn()] = position
    return result


def change_turn(state):
    """ Change the turn 
    
    Parameters
    ----------
    state -- a state
    
    Returns
    --------
    Returns a new state or the same it not possible change turn
    """
    if state.get_players()[state.get_turn()] is None:
        return state
    return State(copy.deepcopy(state.get_players()), copy.deepcopy(state.get_boxes()), copy.deepcopy(state.get_water()), not copy.copy(state.get_turn()))


def is_goal(state):
    """ Change the turn 
    
    Parameters
    ----------
    state -- a state
    
    Returns
    --------
    Returns True if the state is a goal
    """
    return all(player is None for player in state.get_players())

In [ ]:
HTML(value = htmlStr)

In [ ]:
with open("."+os.sep+"levels"+os.sep+"level_medio.txt",'r') as level_file:
    level, state = l.load_level(level_file.read())
state

In [ ]:
# Convertir en pruebas unitarias si se desea
# Use this code to create unit tests

# os.sep es el separador de ficheros ("/" en linux "\" en windows)
level_txt = open("."+os.sep+"levels"+os.sep+"level_medio.txt",'r').read()

level, state = l.load_level(level_txt)
print(state)
s1 = change_turn(state)
print(s1)
s2 = move(level,s1,[0,1])
print(s2)
s3 = move(level,s2,[0,1])
print(s3)
s4 = move(level,s3,[0,1])
print(s4)
s5 = move(level,s4,[-1,0])
print(s5)


'''
Imprimiría
[[3, 4], [2, 1]]{(2, 2)}{(2, 4)}False
[[3, 4], [2, 2]]{(2, 3)}{(2, 4)}False
[[3, 4], [2, 3]]set()set()False
[[3, 4], [2, 4]]set()set()False
[[3, 4], None]set()set()True

'''

Una vez hayas implementado y probado las funciones, trata de ejecutar el juego.

El Mediador, que es la clase que sirve de enganche entre las clases de interfaz y la funcionalidad, leerá las celdas que empiezan por "`# Cargar Celda`", cargará esas funciones dinámicamente, las usará para mover y comprobar si la partida finaliza.


# Atención
El código de abajo carga el notebook actual, por lo que si queremos que el funcionamiento se actualize debemos guardar el notebook y recargar todo el proyecto.

In [1]:
from IPython.display import display
from Mediador import Mediator
from UI import gui
ui = gui()

import warnings
warnings.filterwarnings("ignore")

med = Mediator.get_instance(modelPath = "Kwirk_2.ipynb")   
med.register_ui(ui)


# Cuando llamo a la función se crean
display(ui.get_ui_elements())